Following Free Code Camps tutorial closely.

In [1]:
import ollama
import numpy as np
import torch

In [2]:
question = "What is a Kalman filter"
document = "Chapter 5: AR and Kalman Filters"

In [3]:
import tiktoken

def num_tokens_from_str(string: str, encoding_name: str) -> int:
    """Returns number of tokens in a text string"""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return encoding.encode(string), num_tokens

In [4]:
num_tokens_from_str(question, "cl100k_base")

([3923, 374, 264, 27930, 1543, 4141], 6)

In [ ]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

embeddings = OllamaEmbeddings(model="llama3.2")

query_result = embeddings.embed_query(question)
document_result = embeddings.embed_query(document)

In [6]:
print(str(query_result)[:100])
print(len(query_result))

[-0.0073224027, -0.019477012, 0.017059486, -0.0046530142, 0.0021997404, -0.009726169, 0.008544929, -
3072


In [7]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product/(norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print(similarity)

0.6177236244940539


In [48]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Lecture 1 Magnetic fields and magnetic force.pdf")
loader2 = PyPDFLoader("Lecture 7- The MOS transistors.pdf")
docs = loader.load()
docs2 = loader2.load()

print(f"Total characters: {len(docs[2].page_content)}")
print(f"Total characters: {len(docs2[5].page_content)}")
print(len(docs2))
print(docs[5].page_content)

Total characters: 1647
Total characters: 659
42
PHYS2003: Physics for Electrical Engineers, semester 1 2023 
Lecture 1 
6 
 
 
Figure 3: Simplified diagram of the J. J. Thomson experiment that discovered the electron. 
5.2. The Hall Effect 
In a conductor, when there is no current through it, the electrons move randomly with no 
net or average motion in any direction. When a current is passed through the conductor, 
the electrons continue to move randomly, but with an average drift in the opposite 
direction to the electric field causing the current. The drift speed 𝑣𝑑 is very smaller 
compared with the speed of the electrons in their individual random motion. In a copper 
wire at room temperature, the electrons are randomly moving at around 106 m/s, while the 
drift speed is only around 10-4 m/s (i.e. on the order of centimetres per hour). 
In 1879, Edwin H. Hall showed that the drifting conduction electrons in a copper wire can be 
deflected by an outside magnetic field. This is the 

In [49]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

all_docs = docs + docs2

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 300,
    chunk_overlap=50
)

#Make splits
splits = text_splitter.split_documents(all_docs)

In [ ]:
vectorstore = InMemoryVectorStore.from_documents(splits, embedding=embeddings)

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

From RAG Tutorial this appears to end the embedding and indexing section, next video we will see retrieval

# Part3 - Retrieval

In this section code camp goes over the retrieval section

In [12]:
docsearch = retriever.get_relevant_documents("How does an NMOS Transistor work")

/var/folders/xc/68d5fbnj61b_bgvbkv5gj8t40000gn/T/ipykernel_80254/3037671115.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docsearch = retriever.get_relevant_documents("How does an NMOS Transistor work")


In [13]:
docsearch2 = retriever.get_relevant_documents("List the basic axioms of circuit theory")

In [30]:
print(docsearch2)

[Document(id='adaf80e3-d555-4ebd-961f-95703746a7fa', metadata={'producer': 'macOS Version 12.7.6 (Build 21H1320) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-02-23T10:04:53+08:00', 'author': 'Wen', 'moddate': "D:20250311071035Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 1 - Basic circuit theory', 'source': 'Lecture 1 - Basic circuit theory.pdf', 'total_pages': 36, 'page': 0, 'page_label': '1'}, page_content='Lecture 1\nBasic circuit theory'), Document(id='7aeba4a7-9109-4459-9a05-471271c06d7a', metadata={'producer': 'macOS Version 12.7.6 (Build 21H1320) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-02-23T10:04:53+08:00', 'author': 'Wen', 'moddate': "D:20250311071035Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 1 - Basic circuit theory', 'source': 'Lecture 1 - Basic circuit theory.pdf', 'total_pages': 36, 'page': 19, 'page_label': '20'}, page_content='20\nLinear circuit e

In [31]:
print(docsearch)

[Document(id='805e7137-9b0c-48f2-aaa4-cbd20efbbaff', metadata={'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': "D:20240405072534Z00'00'", 'author': 'Wen', 'moddate': "D:20250521165548Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 7- The MOS transistors [Compatibility Mode]', 'source': 'Lecture 7- The MOS transistors.pdf', 'total_pages': 42, 'page': 2, 'page_label': '3'}, page_content='3\n50nm transistor dimension\n1.2nm Gate \nOxide\nSD\n~2000x smaller than diameter of human hair\nIntel 50nm transistor in production'), Document(id='adaf80e3-d555-4ebd-961f-95703746a7fa', metadata={'producer': 'macOS Version 12.7.6 (Build 21H1320) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2024-02-23T10:04:53+08:00', 'author': 'Wen', 'moddate': "D:20250311071035Z00'00'", 'title': 'Microsoft PowerPoint - Lecture 1 - Basic circuit theory', 'source': 'Lecture 1

In [53]:
docsearch3 = retriever.get_relevant_documents("What is the Hall effect")

print(docsearch3)

[Document(id='ebf7a57c-39d3-4d5a-99a5-94e17f58c10b', metadata={'producer': 'macOS Version 12.5 (Build 21G72) Quartz PDFContext, AppendMode 1.1', 'creator': 'Microsoft® Word 2016', 'creationdate': "D:20230224061506Z00'00'", 'author': 'David Gozzard', 'moddate': "D:20230305080614Z00'00'", 'source': 'Lecture 1 Magnetic fields and magnetic force.pdf', 'total_pages': 7, 'page': 5, 'page_label': '6'}, page_content='voltage is proportional to the strength of the magnetic field. Such a Hall effect sensor (or \nHall sensor) has many applications. \nHall effect sensors can detect both the magnitude and orientation of a magnetic field, and \nthey can work with static (unchanging) magnetic fields. They have no moving parts, so are \nnot prone to wear, and can operate at much higher speeds than mechanical switches. Hall'), Document(id='513bd17d-3214-4086-bf90-aab7cc80afa7', metadata={'producer': 'macOS Version 15.4.1 (Build 24E263) Quartz PDFContext, AppendMode 1.1', 'creator': 'PScript5.dll Versio

In [58]:
print(docsearch3[0].page_content)

voltage is proportional to the strength of the magnetic field. Such a Hall effect sensor (or 
Hall sensor) has many applications. 
Hall effect sensors can detect both the magnitude and orientation of a magnetic field, and 
they can work with static (unchanging) magnetic fields. They have no moving parts, so are 
not prone to wear, and can operate at much higher speeds than mechanical switches. Hall


In [69]:
docsearch4 = retriever.get_relevant_documents("What did Edwin H. Hall discover")

print(docsearch4[0].page_content)

sensor technology. 
Imagine a copper strip with current flowing down the page. The electrons then drift up the 
page. If we apply a magnetic field pointing into the page, the force due to the magnetic field 
will deflect the electrons to the right-hand edge of the strip. 
This caused a greater density of electrons at the right-hand edge of the strip, and so a 
separation of charges that produces an electric field within the strip. The force on the 
electrons due to the electric field (equivalently, their own mutual repulsion) will eventually 
cancel the force due to the magnetic field, resulting in a maximum value of the electric field. 
The electric field results in a potential difference across the strip, which can be measured 
with a voltmeter. 
In addition to allowing us to determine things such as whether the moving charges within a 
conductor are positive or negative, and measure the drift speed of electrons (if we move the 
strip through the magnetic field with same speed as the

# Part 4 - Generation

In [59]:
from langchain.prompts import PromptTemplate

template = """Answer the following question only using the following context:
{context}

If the answer is not contained in the context, respond with:
"I cannot answer this question because the necessary information was not found in the provided documents."

When answering, include the **source file name** and **slide/page number** if available.

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the following question only using the following context:\n{context}\n\nIf the answer is not contained in the context, respond with:\n"I cannot answer this question because the necessary information was not found in the provided documents."\n\nWhen answering, include the **source file name** and **slide/page number** if available.\n\nQuestion: {question}\n')

In [60]:
llm = OllamaLLM(model = "llama3.2")

In [61]:
chain = prompt | llm

In [ ]:
print(chain.invoke({"context":docsearch3, "question":"What is the Hall effect?"}))

The Hall effect is a phenomenon where voltage is proportional to the strength of the magnetic field. This concept is discussed in the document "Lecture 1 Magnetic fields and magnetic force.pdf", page 5, slide 6. According to this document, a Hall effect sensor (or Hall sensor) has many applications, including detecting both the magnitude and orientation of a magnetic field, and can work with static magnetic fields.


In [73]:
print(chain.invoke({"context":docsearch4, "question":"Use an example for drift velocity"}))

The drift velocity of electrons can be used as an example to demonstrate the Hall effect. 

Consider a copper strip with current flowing down the page. The electrons then drift up the page due to the electric field generated by the magnetic field (Hall effect). If we apply a magnetic field pointing into the page, the force on the electrons due to the magnetic field will deflect them to the right-hand edge of the strip.

For example, let's assume that the Hall effect sensor is detecting a magnetic field strength of 0.5 T. According to the Hall effect equation, the voltage output (V) of the Hall sensor is proportional to the product of the magnetic field strength (B), the current density (J), and the thickness of the conductor (t):

V ∝ B × J × t

If we assume that the drift velocity of electrons is approximately 10^4 m/s, and the thickness of the copper strip is 1 μm, we can estimate the Hall voltage as:

V ≈ B × J × t
= 0.5 T × (drift velocity / 1 μm) × 1 μm
≈ 50 V

This example illust

In [28]:
print(chain.invoke({"context":all_docs, "question":"Explain what happens when there is negative gate voltage"}))

According to the document, when there is a negative gate voltage, it means that the gate-to-source potential difference is zero. This results in a channel not existing between the source and drain, as described on page 40 of the lecture notes.

In other words, with a negative gate voltage, the MOSFET is in the "cut-off" or "OFF" region, meaning current cannot pass between the source and drain.


In [ ]:
"""def ask(question, k = 3):
    retrieved_docs = retriever.get_relevant_documents(question)

    # Combine page content into context string
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # Ask the model using your RAG prompt
    response = chain.invoke({"context": context, "question": question})

    # Optionally print sources
    sources = [doc.metadata.get("source", "unknown") + f" (page {doc.metadata.get('page_label', '?')})" for doc in retrieved_docs]
    
    print(f"\n📌 Question: {question}\n🧠 Answer:\n{response}\n📄 Sources: {sources}")
    return response"""


#The above is a function that should return an LLM output given a prompt

'def ask(question, k = 3):\n    retrieved_docs = retriever.get_relevant_documents(question)\n\n    # Combine page content into context string\n    context = "\n\n".join([doc.page_content for doc in retrieved_docs])\n\n    # Ask the model using your RAG prompt\n    response = chain.invoke({"context": context, "question": question})\n\n    # Optionally print sources\n    sources = [doc.metadata.get("source", "unknown") + f" (page {doc.metadata.get(\'page_label\', \'?\')})" for doc in retrieved_docs]\n\n    print(f"\n📌 Question: {question}\n🧠 Answer:\n{response}\n📄 Sources: {sources}")\n    return response'


📌 Question: Explain the operation principle
🧠 Answer:
**Source File Name:** Slide 37 (Hall Effect)
**Slide/Page Number:** Not specified

The Hall effect sensor operates on the principle that a magnetic field deflects electrons in a conductor, creating a separation of charges and an electric field. When a copper strip with current flowing down the page is placed in a magnetic field pointing into the page, the force due to the magnetic field deflects the electrons to the right-hand edge of the strip, resulting in a greater density of electrons at that edge.

As a result, a separation of charges occurs within the strip, producing an electric field. The force on the electrons due to the electric field eventually cancels out the force due to the magnetic field, leading to a maximum value of the electric field.

The Hall effect sensor measures this maximum electric field, which results in a potential difference across the strip. This potential difference is proportional to the strength of t

'**Source File Name:** Slide 37 (Hall Effect)\n**Slide/Page Number:** Not specified\n\nThe Hall effect sensor operates on the principle that a magnetic field deflects electrons in a conductor, creating a separation of charges and an electric field. When a copper strip with current flowing down the page is placed in a magnetic field pointing into the page, the force due to the magnetic field deflects the electrons to the right-hand edge of the strip, resulting in a greater density of electrons at that edge.\n\nAs a result, a separation of charges occurs within the strip, producing an electric field. The force on the electrons due to the electric field eventually cancels out the force due to the magnetic field, leading to a maximum value of the electric field.\n\nThe Hall effect sensor measures this maximum electric field, which results in a potential difference across the strip. This potential difference is proportional to the strength of the magnetic field and can be measured with a vo